# Top 5 Quotes dummy model

In [1]:
import pandas as pd
import itertools
import re
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import string
from sklearn.neighbors import NearestNeighbors
import numpy as np

In [2]:
file = "../raw_data/quotes - reduced.csv"
quotes = pd.read_csv(file, decimal=",")
quotes.head(3)

,quote,author,category
0,"I'm selfish, impatient and a little insecure. ...",Marilyn Monroe,"attributed-no-source, best, life, love, mistak..."
1,You've gotta dance like there's nobody watchin...,William W. Purkey,"dance, heaven, hurt, inspirational, life, love..."
2,You know you're in love when you can't fall as...,Dr. Seuss,"attributed-no-source, dreams, love, reality, s..."


In [3]:
tags_list = quotes.category.to_list()
number_of_tags = []
for i in tags_list:
    number_of_tags.append(len(str(i).split(',')))

In [4]:
quotes['quotes_length'] = quotes['quote'].copy()
for index, row in quotes.iterrows():
    quotes.loc[index, "quotes_length"] = len(str(row['quotes_length']).split(' '))

## Cleaning data: tags are all lowercase, '-' were replaced with ',', duplicates were removed

In [5]:
for index, row in quotes.iterrows():
    quotes.loc[index, "category"] = str(row['category']).lower().replace('-',', ')

In [6]:
quotes['list_tags'] = quotes['category'].copy()
for index,row in quotes.iterrows():
    quotes.loc[index, 'count_tags'] = len(str(row['list_tags']).split(','))
quotes.count_tags.describe()

count    156656.000000
mean          9.747638
std          15.356019
min           1.000000
25%           3.000000
50%           5.000000
75%          11.000000
max         568.000000
Name: count_tags, dtype: float64

In [7]:
for index,row in quotes.iterrows():
    quotes.at[index, 'list_tags'] = str(row['list_tags']).split(',')

In [8]:
for index,row in quotes.iterrows():
    quotes.at[index, 'list_tags'] = str(set(row['list_tags']))

In [9]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text
quotes['list_tags'] = quotes['list_tags'].apply(remove_punctuations)

In [10]:
vectorizer = TfidfVectorizer()
tags_vectorized = vectorizer.fit_transform(quotes['list_tags'])
dummy_tags = LatentDirichletAllocation(n_components=5)
lda_vectors = dummy_tags.fit_transform(tags_vectorized)

In [11]:
def print_topics(model, vectorizer):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names_out()[i], topic[i])
                        for i in topic.argsort()[:-10 - 1:-1]])
        

print_topics(dummy_tags, vectorizer.fit(quotes['list_tags']))

Topic 0:
[('writing', 2092.699680062758), ('love', 1482.2008759826438), ('friendship', 1317.601819132401), ('art', 1278.3097233717438), ('fiction', 1232.399314562284), ('politics', 1141.3054194335484), ('fantasy', 1028.4614209343972), ('relationships', 958.2605934741035), ('time', 675.9473369295404), ('family', 657.3087348974116)]
Topic 1:
[('inspirational', 2749.0811191449657), ('quotes', 1918.4728921933042), ('books', 1603.9010406686964), ('motivational', 1400.15041983004), ('women', 1344.1973908487091), ('reading', 1316.5412642042857), ('life', 1315.241575429486), ('love', 1290.9624057586764), ('romance', 1260.7366653291044), ('feminism', 1211.366521606456)]
Topic 2:
[('humor', 1993.7917482801718), ('wisdom', 1837.4950901673237), ('truth', 1831.2058241002237), ('life', 1808.856862165252), ('education', 1692.9238760028763), ('philosophy', 1542.907086744871), ('knowledge', 1480.8885621696495), ('happiness', 1420.776238184904), ('humour', 1328.1551423457142), ('hope', 1275.976118124566

In [16]:
lda_vectors[1]

array([0.05866288, 0.05911539, 0.37920613, 0.05903043, 0.44398517])

In [13]:
quotes.iloc[-1] = ['me, Paris, selfie, singing, book','image','image','5','image','1']

In [14]:
picture_vectorized = vectorizer.transform(quotes.iloc[-1])
lda_vectors_self = dummy_tags.transform(picture_vectorized)
for index,row in quotes.iterrows():
    quotes['topic'] = quotes.quote.copy()

In [ ]:
for index,row in quotes.iterrows():
    quotes.at[index, 'topic'] = int(np.where(lda_vectors[index] == max(lda_vectors[index]))[0])

__Adding the image caption as the last line to our dataset to be able to see the closest neighbourhs__

In [22]:
quotes.iloc[-1] = ['This is me, I have taken a selfie in Paris.','image','image','10','image','1','?']

In [28]:
quotes.loc[quotes['author'] == 'image', ['topic']] = np.where(lda_vectors[-1] == max(lda_vectors[-1]))[0]


In [29]:
quotes.tail()

,quote,author,category,quotes_length,list_tags,count_tags,topic
156652,…In this way that he sought to control the ver...,"Jesse Ball, The Curfew","life, living, mindfulness",39,mindfulness living life,3.0,[3]
156653,"No matter how we choose to live, we both die a...","Adam Silvera, They Both Die at the End","death, living",13,living death,2.0,[4]
156654,The goal that you hope you will one day arrive...,"Marcus Aurelius, The Essential Marcus Aurelius",living,239,living,1.0,[1]
156655,"This is me, I have taken a selfie in Paris.",image,image,10,image,1,4
-1,NaN,NaN,NaN,NaN,NaN,NaN,[4]
